In [ ]:
##### Coarse grain simulations using HTF #####
## Files needed: PDB file, oplsaa.xml, hoomd_ff.py ##
## Step 1: Create the all atom system using mbuild and dump parameters ##
## Step 2: Build training graph ##
## Step 3: Hoomd-Sim ##
## Codes used in steps 1-3 are in separate files ##

In [ ]:
#### Create all atom system for 256 methanol molecules ####
import hoomd_ff
import gsd,gsd.hoomd, mbuild as mb, pickle
import os

result_dir ='home/CG/'  #path to the results directory

#Create a methanol molecule from a pdb file using mbuild 
class Meth(mb.Compound):
    def __init__(self,N):
        super(Meth, self).__init__()
        mb.load('methanol.pdb', compound=self) 
        self.translate(-self[0].pos)

        self.name = 'Methanol'

N = 256   #number of methanol molecules in the system  
glc_sys = Meth(1)
sys = mb.fill_box(glc_sys, n_compounds= N,density = 787.727, edge = 0.2) #create a box using density and N
sys.translate(-sys.pos)
boxd = sys.periodicity[0]
box = mb.Box(mins=3 * [-boxd/2], maxs = 3*[boxd/2]) #calculate minimum and maximum box dimensions
print('System bounding box is:\n',sys.periodicity)
sys.save(result_dir + 'meth.gsd', overwrite=True)


param_sys, kwargs = hoomd_ff.prepare_hoomd(sys, forcefield_files=['oplsaa.xml'], forcefield_debug=False, box=box) #extract forcefield parameters from oplsaa.xml
mb.formats.gsdwriter.write_gsd(param_sys, result_dir + 'meth.gsd', shift_coords=True, write_special_pairs=True, **kwargs) #write to file in gsd format

with open(result_dir + 'model.p', 'wb') as f:    
    pickle.dump(param_sys, f) #dump model parameters


In [ ]:
### Graph building code ###
import tensorflow as tf
import hoomd.tensorflow_plugin as htf
from tensorflow.keras import layers
import numpy as np

def make_train_graph(model_directory, NN, rcut, NN_aa, N_hidden, system):

    graph = htf.graph_builder(0, output_forces=False) #mamximum number of neighbors=0, graph will not compute forces to use in TF
    assign_ops = []
    prints = []
    molecule_mapping = htf.find_molecules(system) # get mapping from molecule index to particle index

    for m in molecule_mapping:
        assert len(m) == len(molecule_mapping[0]) # ensure all molecules are of the same length
    N = sum([len(m) for m in molecule_mapping])
    M = len(molecule_mapping)
    
    MN = len(molecule_mapping[0]) #number of atoms in a molecule=MN 
    molecule_mapping_matrix = np.ones([1, MN], dtype=np.int) # create one bead mapping. mapping matrices must be created for other types of mappings
    cg_mapping = htf.sparse_mapping([molecule_mapping_matrix for _ in molecule_mapping], molecule_mapping, system=system) #create a mass-weighted M x N mapping operator 
    assert cg_mapping.shape == (M, N)
    mapped_pos = htf.center_of_mass(graph.positions[:,:3], cg_mapping, system, name='com-mapped-positions') #calculate the center of mass of a CG bead
    mapped_nlist = htf.compute_nlist(mapped_pos, rcut, NN, system, False) #create the mapped neighbot list
    force_cg_mapping = htf.sparse_mapping([molecule_mapping_matrix for _ in molecule_mapping], molecule_mapping) #create a non mass-weighted M x N mapping operator 
    mapped_force = tf.sparse.matmul(force_cg_mapping, graph.forces, name='mapped-forces') #calculates the coarse grained forces

    rdf = graph.compute_rdf([0.1,rcut], 'rdf', nbins=200, nlist=mapped_nlist) #create the rdf using mapped neighbot list
    

In [ ]:
### Hoomd-Sim code ###

with open(result_dir + 'model.p', 'rb') as f:
    param_sys = pickle.load(f) #load parameters

g = gsd.hoomd.open(result_dir + 'meth.gsd')
frame = g[0]
set_rcut = 10
c = hoomd.context.initialize('--gpu=1') #Start TensorFlow
system = hoomd.init.read_gsd(filename=result_dir + 'meth.gsd')
c.sorter.disable()

build_htf_models.make_train_graph(model_dir, 128, set_rcut, 816, 48, system) #calls the graph building code

##hoomd_ff.py script is used to set up parameters##
with hoomd.tensorflow_plugin.tfcompute(model_dir) as tfcompute:
    nlist = hoomd.md.nlist.cell()
    pair_forces = hoomd_ff.pair_coeffs(frame, param_sys, nlist, r_cut=set_rcut) #set-up pair forces
    
    #set-up pppm
    charged = hoomd.group.all();
    pppm = hoomd.md.charge.pppm(nlist=nlist, group=charged)
    pppm.set_params(Nx=32, Ny=32, Nz=32, order=6, rcut=set_rcut)

    #set-up bonds
    hoomd_ff.bond_coeffs(frame, system, param_sys)

    #set-up angles
    hoomd_ff.angle_coeffs(frame, param_sys)

    #set-up dihedrals
    hoomd_ff.dihedral_coeffs(frame, param_sys)

    group_all = hoomd.group.all()
    kT = 1.9872/1000

    #NVT Simulation in Hoomd
    im = hoomd.md.integrate.mode_standard(dt=2 / 48.9)
    logger = hoomd.analyze.log(filename= result_dir+'init.log', quantities=['temperature',  'potential_energy','kinetic_energy'], period=10)
    nvt = hoomd.md.integrate.nvt(group=group_all, kT=298.15 * kT, tau=350 / 48.9)
    nvt.randomize_velocities(1234)

    #equilibrate
    hoomd.run(1e4)
    nvt_dump = hoomd.dump.gsd(filename=result_dir+'trajectory.gsd', period=1e3, group=group_all, phase=0, overwrite=True)
    
    tfcompute.attach(period=1, save_period=1e2) #communicate positions, neighbor list and forces to TensorFlow model
    tfcompute.set_reference_forces(*pair_forces, pppm, pppm.ewald)
    
    #Hoomd production run
    hoomd.run(1e4) 
